In [33]:
!pip -q install -U sentence-transformers joblib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 8.1 MB/s eta 0:00:00a 0:00:01


In [34]:
import os, re, json, glob
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from scipy import sparse
import matplotlib.pyplot as plt

# Optional (heatmap)
import seaborn as sns

# Embeddings
from sentence_transformers import SentenceTransformer

2025-12-27 14:16:22.505145: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766844982.705373      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766844982.760727      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766844983.235653      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766844983.235696      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766844983.235699      55 computation_placer.cc:177] computation placer alr

In [35]:
DATA_DIR = "/kaggle/input"
dataset_folder = None
for p in glob.glob(os.path.join(DATA_DIR, "*top-15000*")):
    dataset_folder = p
    break

if dataset_folder is None:
    raise FileNotFoundError("Không tìm thấy folder dataset top-15000 trong /kaggle/input")

csvs = glob.glob(os.path.join(dataset_folder, "*.csv"))
if not csvs:
    # đôi khi dataset có folder con
    csvs = glob.glob(os.path.join(dataset_folder, "**", "*.csv"), recursive=True)

if not csvs:
    raise FileNotFoundError("Không tìm thấy file .csv trong dataset folder.")

csv_path = csvs[0]
print("Using CSV:", csv_path)

df_raw = pd.read_csv(csv_path)
print(df_raw.shape)
df_raw.head(3)

Using CSV: /kaggle/input/top-15000-ranked-anime-dataset-update-to-32025/top_15000_anime.csv
(15000, 24)


,anime_id,anime_url,image_url,name,english_name,japanese_names,score,genres,themes,demographics,...,producers,studios,source,duration,rating,rank,popularity,favorites,scored_by,members
0,52991,https://myanimelist.net/anime/52991/Sousou_no_...,https://cdn.myanimelist.net/images/anime/1015/...,Sousou no Frieren,Frieren: Beyond Journey's End,葬送のフリーレン,9.29,"Adventure, Drama, Fantasy",NaN,Shounen,...,"Aniplex, Dentsu, Shogakukan-Shueisha Productio...",Madhouse,Manga,24 min per ep,PG-13 - Teens 13 or older,1.0,128,76513,734207.0,1225468
1,5114,https://myanimelist.net/anime/5114/Fullmetal_A...,https://cdn.myanimelist.net/images/anime/1208/...,Fullmetal Alchemist: Brotherhood,Fullmetal Alchemist: Brotherhood,鋼の錬金術師 FULLMETAL ALCHEMIST,9.10,"Action, Adventure, Drama, Fantasy",Military,Shounen,...,"Aniplex, Square Enix, Mainichi Broadcasting Sy...",Bones,Manga,24 min per ep,R - 17+ (violence & profanity),2.0,3,236798,2249670.0,3577489
2,9253,https://myanimelist.net/anime/9253/Steins_Gate,https://cdn.myanimelist.net/images/anime/1935/...,Steins;Gate,Steins;Gate,STEINS;GATE,9.07,"Drama, Sci-Fi, Suspense","Psychological, Time Travel",NaN,...,"Frontier Works, Media Factory, Kadokawa Shoten...",White Fox,Visual novel,24 min per ep,PG-13 - Teens 13 or older,3.0,14,198296,1483605.0,2737980


In [36]:
def norm_col(c: str) -> str:
    return re.sub(r"[^a-z0-9]+", "_", str(c).strip().lower())

df = df_raw.copy()
df.columns = [norm_col(c) for c in df.columns]

def pick_col(cols, candidates):
    for c in candidates:
        if c in cols:
            return c
    # fallback: find substring
    for c in cols:
        for key in candidates:
            if key in c:
                return c
    return None

cols = set(df.columns)

c_id   = pick_col(cols, ["anime_id","mal_id","id"])
c_title= pick_col(cols, ["title","name","anime_title"])
c_syn  = pick_col(cols, ["synopsis","description","sypnosis","overview"])
c_gen  = pick_col(cols, ["genres","genre"])
c_type = pick_col(cols, ["type","format"])
c_eps  = pick_col(cols, ["episodes","eps"])
c_score= pick_col(cols, ["score","rating","scored"])
c_scby = pick_col(cols, ["scored_by","score_count","users_scored","members_scored","scorers"])
c_mem  = pick_col(cols, ["members","member","favorites","favourites"])
c_pop  = pick_col(cols, ["popularity","popular"])
c_rank = pick_col(cols, ["rank","ranking"])
c_year = pick_col(cols, ["year","release_year","aired_from_year","start_year"])
c_seas = pick_col(cols, ["season","premiered"])
c_stat = pick_col(cols, ["status"])
c_stu  = pick_col(cols, ["studios","studio","producer"])
c_src  = pick_col(cols, ["source"])
c_dur  = pick_col(cols, ["duration"])
c_img  = pick_col(cols, ["image_url","img_url","image","picture","poster","cover","imageurl"])
c_url  = pick_col(cols, ["url","link"])

print("Mapped columns:")
for k,v in {
    "id":c_id, "title":c_title, "synopsis":c_syn, "genres":c_gen, "type":c_type,
    "episodes":c_eps, "score":c_score, "scored_by":c_scby, "members":c_mem,
    "popularity":c_pop, "rank":c_rank, "year":c_year, "season":c_seas,
    "status":c_stat, "studios":c_stu, "source":c_src, "duration":c_dur,
    "image_url":c_img, "url":c_url
}.items():
    print(f"- {k:10s}: {v}")

Mapped columns:
- id        : anime_id
- title     : name
- synopsis  : synopsis
- genres    : genres
- type      : type
- episodes  : episodes
- score     : score
- scored_by : scored_by
- members   : members
- popularity: popularity
- rank      : rank
- year      : None
- season    : premiered
- status    : None
- studios   : studios
- source    : source
- duration  : duration
- image_url : image_url
- url       : anime_url


In [37]:
def safe_str(x):
    if pd.isna(x):
        return ""
    return str(x).strip()

def safe_num(x):
    try:
        if pd.isna(x): return np.nan
        return float(x)
    except:
        return np.nan

# Build canonical frame
out = pd.DataFrame()
out["anime_id"] = df[c_id] if c_id else np.arange(len(df))
out["title"] = df[c_title].apply(safe_str) if c_title else ""
out["synopsis"] = df[c_syn].apply(safe_str) if c_syn else ""
out["genres"] = df[c_gen].apply(safe_str) if c_gen else ""
out["type"] = df[c_type].apply(safe_str) if c_type else ""
out["episodes"] = df[c_eps].apply(safe_num) if c_eps else np.nan
out["score"] = df[c_score].apply(safe_num) if c_score else np.nan
out["scored_by"] = df[c_scby].apply(safe_num) if c_scby else np.nan
out["members"] = df[c_mem].apply(safe_num) if c_mem else np.nan
out["popularity"] = df[c_pop].apply(safe_num) if c_pop else np.nan
out["rank"] = df[c_rank].apply(safe_num) if c_rank else np.nan
out["year"] = df[c_year].apply(safe_num) if c_year else np.nan
out["season"] = df[c_seas].apply(safe_str) if c_seas else ""
out["status"] = df[c_stat].apply(safe_str) if c_stat else ""
out["studios"] = df[c_stu].apply(safe_str) if c_stu else ""
out["source"] = df[c_src].apply(safe_str) if c_src else ""
out["duration"] = df[c_dur].apply(safe_str) if c_dur else ""
out["image_url"] = df[c_img].apply(safe_str) if c_img else ""
out["url"] = df[c_url].apply(safe_str) if c_url else ""

# Missing values handling
out["title"] = out["title"].replace("", np.nan)
out = out.dropna(subset=["title"]).copy()

# Normalize text
for col in ["title","synopsis","genres","type","season","status","studios","source"]:
    out[col] = out[col].fillna("").astype(str).str.strip()

# Remove duplicates (title)
out = out.drop_duplicates(subset=["title"]).reset_index(drop=True)

# Outlier handling (clip numeric)
for col in ["score","scored_by","members","popularity","rank","episodes","year"]:
    if col in out.columns:
        x = out[col].astype(float)
        lo, hi = np.nanpercentile(x, 1), np.nanpercentile(x, 99)
        out[col] = x.clip(lo, hi)

# Score filled
out["score_filled"] = out["score"].fillna(out["score"].median())

# Weighted score (IMDB-style)
C = out["score_filled"].mean()
v = out["scored_by"].fillna(0.0)
m = np.nanpercentile(v, 90)  # threshold
R = out["score_filled"]
out["weighted_score"] = (v/(v+m))*R + (m/(v+m))*C

# Combined text for vectorization
out["combined_text"] = (
    out["title"] + " " +
    out["synopsis"] + " " +
    out["genres"] + " " +
    out["studios"] + " " +
    out["type"] + " " +
    out["source"] + " " +
    out["season"] + " " +
    out["status"]
).str.replace(r"\s+", " ", regex=True).str.strip()

out.head(3)

/usr/local/lib/python3.12/dist-packages/numpy/lib/_nanfunctions_impl.py:1424: RuntimeWarning: All-NaN slice encountered
  return _nanquantile_unchecked(


,anime_id,title,synopsis,genres,type,episodes,score,scored_by,members,popularity,...,season,status,studios,source,duration,image_url,url,score_filled,weighted_score,combined_text
0,52991,Sousou no Frieren,During their decade-long quest to defeat the D...,"Adventure, Drama, Fantasy",TV,28.0,8.5,641392.95,1112025.75,150.95,...,Fall 2023,,Madhouse,Manga,24 min per ep,https://cdn.myanimelist.net/images/anime/1015/...,https://myanimelist.net/anime/52991/Sousou_no_...,8.5,8.289015,Sousou no Frieren During their decade-long que...
1,5114,Fullmetal Alchemist: Brotherhood,After a horrific alchemy experiment goes wrong...,"Action, Adventure, Drama, Fantasy",TV,64.0,8.5,641392.95,1112025.75,150.95,...,Spring 2009,,Bones,Manga,24 min per ep,https://cdn.myanimelist.net/images/anime/1208/...,https://myanimelist.net/anime/5114/Fullmetal_A...,8.5,8.289015,Fullmetal Alchemist: Brotherhood After a horri...
2,9253,Steins;Gate,Eccentric scientist Rintarou Okabe has a never...,"Drama, Sci-Fi, Suspense",TV,24.0,8.5,641392.95,1112025.75,150.95,...,Spring 2011,,White Fox,Visual novel,24 min per ep,https://cdn.myanimelist.net/images/anime/1935/...,https://myanimelist.net/anime/9253/Steins_Gate,8.5,8.289015,Steins;Gate Eccentric scientist Rintarou Okabe...


In [38]:
os.makedirs("artifacts/figures", exist_ok=True)

# 1) Score distribution (hist)
plt.figure()
out["score_filled"].plot(kind="hist", bins=30)
plt.title("Score Distribution")
plt.xlabel("score")
plt.ylabel("count")
plt.tight_layout()
plt.savefig("artifacts/figures/score_distribution.png", dpi=160)
plt.close()

# 2) Top genres frequency (barh)
def explode_genres(s):
    if not s:
        return []
    # nhiều dataset dùng "," hoặc "|" hoặc ";"
    s = str(s)
    if "|" in s: parts = s.split("|")
    elif ";" in s: parts = s.split(";")
    else: parts = s.split(",")
    return [g.strip() for g in parts if g.strip()]

genre_series = out["genres"].apply(explode_genres).explode()
genre_counts = genre_series.value_counts().head(20)

plt.figure()
genre_counts.sort_values().plot(kind="barh")
plt.title("Top 20 Genres Frequency")
plt.xlabel("count")
plt.tight_layout()
plt.savefig("artifacts/figures/top_genres.png", dpi=160)
plt.close()

# 3) Top items by weighted score (barh)
top_items = out.sort_values("weighted_score", ascending=False).head(20)
plt.figure(figsize=(8, 6))
plt.barh(top_items["title"][::-1], top_items["weighted_score"][::-1])
plt.title("Top 20 Anime by Weighted Score")
plt.xlabel("weighted_score")
plt.tight_layout()
plt.savefig("artifacts/figures/top_weighted_score.png", dpi=160)
plt.close()

# 4) Correlation heatmap
num_cols = [c for c in ["score_filled","scored_by","members","popularity","rank","episodes","year","weighted_score"] if c in out.columns]
corr = out[num_cols].corr()

plt.figure(figsize=(10, 7))
sns.heatmap(corr, annot=True, fmt=".2f", square=True, linewidths=0.5)
plt.title("Correlation Heatmap (Numeric Features)")
plt.tight_layout()
plt.savefig("artifacts/figures/correlation_heatmap.png", dpi=180)
plt.close()

print("✅ Saved EDA figures to artifacts/figures/")
print("Numeric cols used:", num_cols)

✅ Saved EDA figures to artifacts/figures/
Numeric cols used: ['score_filled', 'scored_by', 'members', 'popularity', 'rank', 'episodes', 'year', 'weighted_score']


In [39]:
tfidf = TfidfVectorizer(
    max_features=50000,
    ngram_range=(1,2),
    min_df=2,
    max_df=0.9,
    stop_words="english"
)

tfidf_matrix = tfidf.fit_transform(out["combined_text"])
print("TFIDF shape:", tfidf_matrix.shape)

os.makedirs("artifacts", exist_ok=True)
sparse.save_npz("artifacts/tfidf_matrix.npz", tfidf_matrix)

import joblib
joblib.dump(tfidf, "artifacts/tfidf_vectorizer.joblib")

TFIDF shape: (14996, 50000)


['artifacts/tfidf_vectorizer.joblib']

In [40]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"
sbert = SentenceTransformer(model_name)

# encode
emb = sbert.encode(
    out["combined_text"].tolist(),
    batch_size=128,
    show_progress_bar=True,
    normalize_embeddings=True
).astype(np.float32)

print("Embeddings:", emb.shape, emb.dtype)

np.save("artifacts/sbert_embeddings.npy", emb)

with open("artifacts/embedding_meta.json", "w", encoding="utf-8") as f:
    json.dump({"model_name": model_name, "dim": int(emb.shape[1])}, f, ensure_ascii=False, indent=2)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/118 [00:00<?, ?it/s]

Embeddings: (14996, 384) float32


In [41]:
# -------------------------
# A) Predict score (RMSE/MAE) - regression proxy
# -------------------------
feat_cols = [c for c in ["scored_by","members","popularity","rank","episodes","year"] if c in out.columns]
X = out[feat_cols].fillna(0.0).astype(float)
y = out["score_filled"].astype(float)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

reg = Pipeline([
    ("scaler", StandardScaler()),
    ("ridge", Ridge(alpha=2.0, random_state=42))
])
reg.fit(X_train, y_train)
pred = reg.predict(X_test)

rmse = float(np.sqrt(mean_squared_error(y_test, pred)))
mae = float(mean_absolute_error(y_test, pred))

print("RMSE:", rmse, "MAE:", mae)

# -------------------------
# B) Ranking eval (Precision@K / Recall@K) using pseudo-relevance
#   Relevant = share >=1 genre with query item
# -------------------------
def get_genre_set(s):
    return set(explode_genres(s))

genre_sets = out["genres"].apply(get_genre_set).tolist()

def precision_recall_at_k(reco_indices, rel_set, k):
    topk = reco_indices[:k]
    hit = sum(1 for i in topk if i in rel_set)
    prec = hit / max(k, 1)
    rec = hit / max(len(rel_set), 1)
    return prec, rec

def topk_tfidf(idx, k=20):
    q = tfidf_matrix[idx]
    sims = (tfidf_matrix @ q.T).toarray().ravel()  # dot product
    sims[idx] = -1
    return np.argsort(-sims)[:k]

def topk_sbert(idx, k=20):
    q = emb[idx]
    sims = emb @ q  # cosine because normalized
    sims[idx] = -1
    return np.argsort(-sims)[:k]

# sample queries for speed
np.random.seed(42)
query_ids = np.random.choice(len(out), size=min(400, len(out)), replace=False)

Ks = [5, 10, 20]
metrics = {"rmse": rmse, "mae": mae, "precision_at_k": {}, "recall_at_k": {}}

for name, topk_fn in [("tfidf", topk_tfidf), ("sbert", topk_sbert)]:
    p = {k: [] for k in Ks}
    r = {k: [] for k in Ks}
    for qi in query_ids:
        qg = genre_sets[qi]
        if len(qg) == 0:
            continue
        rel = {j for j in range(len(out)) if j != qi and len(qg & genre_sets[j]) >= 1}
        if len(rel) == 0:
            continue
        recos = topk_fn(qi, k=max(Ks))
        for k in Ks:
            pk, rk = precision_recall_at_k(recos, rel, k)
            p[k].append(pk)
            r[k].append(rk)
    metrics["precision_at_k"][name] = {str(k): float(np.mean(p[k])) for k in Ks}
    metrics["recall_at_k"][name] = {str(k): float(np.mean(r[k])) for k in Ks}

metrics

RMSE: 0.4359861231672205 MAE: 0.30974233639976684


{'rmse': 0.4359861231672205,
 'mae': 0.30974233639976684,
 'precision_at_k': {'tfidf': {'5': 0.7319587628865979,
   '10': 0.7067010309278352,
   '20': 0.6823453608247423},
  'sbert': {'5': 0.7711340206185566,
   '10': 0.7332474226804123,
   '20': 0.6921391752577318}},
 'recall_at_k': {'tfidf': {'5': 0.00142965095521254,
   '10': 0.0026899877660009446,
   '20': 0.004990827252191299},
  'sbert': {'5': 0.0013405696878658468,
   '10': 0.0025191539725020144,
   '20': 0.004555234003108773}}}

In [42]:
# Save clean data
out.to_csv("artifacts/anime_clean.csv", index=False)

# title/id mapping
title_to_idx = {t: int(i) for i, t in enumerate(out["title"].tolist())}
id_to_idx = {str(out["anime_id"].iloc[i]): int(i) for i in range(len(out))}

with open("artifacts/title_to_idx.json", "w", encoding="utf-8") as f:
    json.dump(title_to_idx, f, ensure_ascii=False)

with open("artifacts/id_to_idx.json", "w", encoding="utf-8") as f:
    json.dump(id_to_idx, f, ensure_ascii=False)

with open("artifacts/metrics.json", "w", encoding="utf-8") as f:
    json.dump(metrics, f, ensure_ascii=False, indent=2)

print("✅ Saved artifacts/")
print(sorted(glob.glob("artifacts/*"))[:20])

✅ Saved artifacts/
['artifacts/anime_clean.csv', 'artifacts/embedding_meta.json', 'artifacts/figures', 'artifacts/id_to_idx.json', 'artifacts/metrics.json', 'artifacts/sbert_embeddings.npy', 'artifacts/tfidf_matrix.npz', 'artifacts/tfidf_vectorizer.joblib', 'artifacts/title_to_idx.json']
